In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from skl2onnx.helpers.onnx_helper import load_onnx_model
from stitchnet.stitchonnx.utils import Net
from glob import glob
from pathlib import Path

netsFiles = sorted(glob('_models/fragments/net*'))
nets = []
for i,netsFile in enumerate(netsFiles):
    fragmentFiles = sorted(glob(str(Path(netsFile)/'fragment*.onnx')))
    onnxFragments = []
    for fragmentFile in fragmentFiles:
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
    net1 = Net(onnxFragments, i)
    nets.append(net1)

/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# import torch
# torch.ones(1,2).cuda()

In [4]:
from stitchnet.stitchonnx.utils import load_cats_and_dogs_dset,convert_imagenet_to_cat_dog_label
from stitchnet.stitchonnx.utils import accuracy_score_model,accuracy_score_net,load_dl
from stitchnet.stitchonnx.utils import generate_networks, ScoreMapper
from stitchnet.stitchonnx.report import Report
from stitchnet.stitchonnx.utils import evalulate_stitchnet

from tqdm import tqdm
import torch
import numpy as np
import os
from collections import defaultdict
import hashlib
import random
import time

random.seed(51)
np.random.seed(24)
torch.manual_seed(77)

STITCH_BATCH_SIZE = 32 # todo study the effect
MAX_DEPTH = 10
THRESOULD = 0.7
TOTAL_THRESOULD = 0.5

RESULT_NAME = f"{int(time.time())}_result_BS_{STITCH_BATCH_SIZE}_MD_{MAX_DEPTH}_T_{THRESOULD}_TT_{TOTAL_THRESOULD}"

EVAL_BATCH_SIZE = 64

dataset_train = load_cats_and_dogs_dset("train")
dataset_val = load_cats_and_dogs_dset("test")

dl_score = load_dl(dataset_train, STITCH_BATCH_SIZE)
data_score,t = next(iter(dl_score))
data_score = data_score.numpy()

In [5]:
# range(1)
k = 0
if os.path.exists(f'./_results/{RESULT_NAME}.txt'):
    with open(f'./_results/{RESULT_NAME}.txt', 'r') as f:
        k = len(f.read().split('\n'))

In [ ]:
from tqdm import tqdm

scoreMapper = ScoreMapper(nets, data_score)
with Report(EVAL_BATCH_SIZE, f'./_results/{RESULT_NAME}.txt', 'w') as report:
    # for _ in tqdm(range(50)):
    generator = generate_networks(nets, scoreMapper, data_score, 
                          threshold=THRESOULD, totalThreshold=TOTAL_THRESOULD, 
                          maxDepth=MAX_DEPTH, sample=False)
    for i,(s,net) in enumerate(generator):
        try:
            netname = f"_results/{RESULT_NAME}/net{k:03}"
            report.evaluate(nets, net, netname, s, dataset_val)
            net.save(netname)
            k += 1
        except Exception as e:
            print('ERROR', e)
            pass

current depth: 1
potential next fragments: 43
potential next fragments after thresholding of 0.7: 1 ['1.0']
totalscore 1.0000001192092896
current depth: 2
potential next fragments: 43
potential next fragments after thresholding of 0.7: 1 ['1.0']
totalscore 1.0000001192092896
current depth: 3
potential next fragments: 43
potential next fragments after thresholding of 0.7: 1 ['1.0']
totalscore 1.0000001192092896
current depth: 4
potential next fragments: 43
potential next fragments after thresholding of 0.7: 3 ['1.0', '0.7', '0.7']
totalscore 1.0000001192092896
current depth: 5
potential next fragments: 43
potential next fragments after thresholding of 0.7: 40 ['0.95', '0.88', '0.88', '0.86', '0.85', '0.84', '0.84', '0.84', '0.83', '0.82', '0.82', '0.82', '0.81', '0.8', '0.8', '0.79', '0.79', '0.79', '0.79', '0.79', '0.79', '0.79', '0.78', '0.78', '0.78', '0.78', '0.77', '0.77', '0.77', '0.76', '0.76', '0.76', '0.75', '0.75', '0.75', '0.74', '0.74', '0.71', '0.71', '0.71']
totalscore 0.9

100%|██████████| 80/80 [00:06<00:00, 12.74it/s]


accuracy 0.7581532416502947
saving to _results/1660390414_result_BS_32_MD_10_T_0.7_TT_0.5/net000
totalscore 0.6589146976100261


100%|██████████| 80/80 [00:06<00:00, 12.13it/s]


accuracy 0.7986247544204322
saving to _results/1660390414_result_BS_32_MD_10_T_0.7_TT_0.5/net001
totalscore 0.5622431693387102


...

100%|██████████| 80/80 [00:08<00:00,  9.61it/s]


accuracy 0.7640471512770137
saving to _results/1660390414_result_BS_32_MD_10_T_0.7_TT_0.5/net182
totalscore 0.5222143899207634


100%|██████████| 80/80 [00:08<00:00,  9.37it/s]


accuracy 0.7707269155206287
saving to _results/1660390414_result_BS_32_MD_10_T_0.7_TT_0.5/net183
totalscore 0.5592239359959374
current depth: 8
potential next fragments: 43
potential next fragments after thresholding of 0.7: 27 ['0.97', '0.93', '0.88', '0.81', '0.8', '0.79', '0.77', '0.77', '0.77', '0.76', '0.76', '0.76', '0.75', '0.75', '0.74', '0.72', '0.72', '0.72', '0.72', '0.72', '0.71', '0.71', '0.71', '0.71', '0.71', '0.7', '0.7']
totalscore 0.543611899349208
current depth: 9
potential next fragments: 5
potential next fragments after thresholding of 0.7: 4 ['0.91', '0.82', '0.77', '0.77']
totalscore 0.5198881367856137


 78%|███████▊  | 62/80 [00:06<00:01, 13.87it/s]

In [ ]:
# dataset_val